### Importando bibliotecas

In [1]:
# desabilita os warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

import plotly.express as px
import matplotlib.pyplot as plt


# from datetime import datetime as dt

### Importando dados tratados

In [3]:
df_solar_wind = pd.read_parquet(r"..\data\df_solar_wind.parquet")
df_satellite_pos = pd.read_parquet(r"..\data\df_satellite_pos.parquet")
df_sunspots = pd.read_parquet(r"..\data\df_sunspots.parquet")
df_labels = pd.read_parquet(r"..\data\df_labels.parquet")


### Consolidando base de dados

##### Preparando dataframes

In [4]:
df_solar_wind["dia"] = df_solar_wind["timedelta"].dt.days
df_solar_wind["dia"] = df_solar_wind["dia"].apply(pd.to_numeric, downcast ="integer")

df_satellite_pos["dia"] = df_satellite_pos["timedelta"].dt.days
df_satellite_pos["dia"] = df_satellite_pos["dia"].apply(pd.to_numeric, downcast ="integer")

In [5]:
df_satellite_pos = pd.merge(df_satellite_pos, df_solar_wind[["period", "dia", "source"]], on=["period", "dia"], how="left")

In [6]:
df_satellite_pos["gse_x_satellite"] = np.nan
df_satellite_pos["gse_y_satellite"] = np.nan
df_satellite_pos["gse_z_satellite"] = np.nan
df_satellite_pos.head(3)

,period,timedelta,gse_x_ace,gse_y_ace,gse_z_ace,gse_x_dscovr,gse_y_dscovr,gse_z_dscovr,dia,source,gse_x_satellite,gse_y_satellite,gse_z_satellite
0,train_a,0 days,1522376.9,143704.6,149496.7,NaN,NaN,NaN,0,ac,NaN,NaN,NaN
1,train_a,0 days,1522376.9,143704.6,149496.7,NaN,NaN,NaN,0,ac,NaN,NaN,NaN
2,train_a,0 days,1522376.9,143704.6,149496.7,NaN,NaN,NaN,0,ac,NaN,NaN,NaN


In [7]:
df_satellite_pos["gse_x_satellite"] = np.where(df_satellite_pos["gse_x_satellite"].isnull() & (df_satellite_pos["source"]  == "ac"),
                                               df_satellite_pos["gse_x_ace"],
                                               df_satellite_pos["gse_x_dscovr"]
                                               )
df_satellite_pos["gse_y_satellite"] = np.where(df_satellite_pos["gse_y_satellite"].isnull() & (df_satellite_pos["source"]  == "ac"),
                                               df_satellite_pos["gse_y_ace"],
                                               df_satellite_pos["gse_y_dscovr"]
                                               )
df_satellite_pos["gse_z_satellite"] = np.where(df_satellite_pos["gse_z_satellite"].isnull() & (df_satellite_pos["source"]  == "ac"),
                                               df_satellite_pos["gse_z_ace"],
                                               df_satellite_pos["gse_z_dscovr"]
                                               )

df_satellite_pos = df_satellite_pos[['period', 
                                     'timedelta', 
                                     'dia', 
                                     'source',
                                     'gse_x_satellite', 
                                     'gse_y_satellite', 
                                     'gse_z_satellite']]

df_satellite_pos["gse_x_satellite"] = df_satellite_pos["gse_x_satellite"].interpolate(method='linear', limit_direction='both')
df_satellite_pos["gse_y_satellite"] = df_satellite_pos["gse_y_satellite"].interpolate(method='linear', limit_direction='both')
df_satellite_pos["gse_z_satellite"] = df_satellite_pos["gse_z_satellite"].interpolate(method='linear', limit_direction='both')

df_satellite_pos["gse_x_satellite"] = df_satellite_pos["gse_x_satellite"].apply(pd.to_numeric, downcast ="float")
df_satellite_pos["gse_y_satellite"] = df_satellite_pos["gse_y_satellite"].apply(pd.to_numeric, downcast ="float")
df_satellite_pos["gse_z_satellite"] = df_satellite_pos["gse_z_satellite"].apply(pd.to_numeric, downcast ="float")

df_satellite_pos.isna().sum()

period             0
timedelta          0
dia                0
source             0
gse_x_satellite    0
gse_y_satellite    0
gse_z_satellite    0
dtype: int64

##### Consolidando dataframes

In [8]:
df_solar_wind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8392320 entries, 0 to 8392319
Data columns (total 11 columns):
 #   Column       Dtype          
---  ------       -----          
 0   period       category       
 1   timedelta    timedelta64[ns]
 2   bx_gse       float32        
 3   by_gse       float32        
 4   bz_gse       float32        
 5   bt           float32        
 6   density      float32        
 7   speed        float32        
 8   temperature  float32        
 9   source       category       
 10  dia          int64          
dtypes: category(2), float32(7), int64(1), timedelta64[ns](1)
memory usage: 368.2 MB


In [9]:
df_satellite_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8392320 entries, 0 to 8392319
Data columns (total 7 columns):
 #   Column           Dtype          
---  ------           -----          
 0   period           category       
 1   timedelta        timedelta64[ns]
 2   dia              int64          
 3   source           category       
 4   gse_x_satellite  float64        
 5   gse_y_satellite  float64        
 6   gse_z_satellite  float64        
dtypes: category(2), float64(3), int64(1), timedelta64[ns](1)
memory usage: 336.1 MB


In [10]:
df_satellite_pos = df_satellite_pos[["period", 
                                     "dia", 
                                     "source", 
                                     'gse_x_satellite', 
                                     'gse_y_satellite', 
                                     'gse_z_satellite']]

In [11]:
df_consolidado = pd.merge(df_solar_wind, df_satellite_pos, on=["period", 
                                                               "dia",
                                                               "source"], 
                                                           how="left")

df_consolidado

MemoryError: Unable to allocate 89.4 GiB for an array with shape (11996502324,) and data type int64